In [1]:
%%capture
!python3 -m pip install geopy map_network_viz --upgrade

In [4]:
import osmnx as ox
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.lines import Line2D
from geopy import geocoders
import matplotlib.pyplot as plt
import re
from pop_us_cities import *
from map_network_viz import geocode


In [5]:
distance = 3000
city_street_dict = {}
for city in us_cities:
    street_names = set()
    latitude, longitude = geocode(city)
    G = ox.graph_from_point((latitude, longitude), distance, network_type="all", retain_all=True, simplify=False)
    H = ox.get_undirected(G)
    for _, edge in ox.graph_to_gdfs(H, nodes=False).fillna('').iterrows():
        if edge['name'] == '':
            continue
        elif edge['name'].split()[0] in ['North', 'South', 'East', 'West', 'Northeast', 'Southeast', 'Northwest', 'Southwest'] and len(edge['name'].split()) == 2:
            street_names.add(edge['name'])
        else:
            street_name = edge['name'].replace('North ', '').replace('South ', '').replace('East ', '').replace('West ', '') \
                .replace('Northeast ', '').replace('Southeast ', '').replace('Northwest ', '').replace('Southwest ', '')
            street_names.add(street_name)
    city_street_dict[city] = street_names

In [9]:
from pprint import pprint
inverted_dict = {}
for city, streets in city_street_dict.items():
    for street in streets:
        inverted_dict.setdefault(street, []).append(city)

sorted_dict = sorted(inverted_dict.items(), key=lambda item: len(item[1]), reverse=True)
pprint([(x[0], len(x[1])) for x in sorted_dict if not any(char.isdigit() for char in x[0])][:10])

[('Main Street', 25),
 ('Pine Street', 20),
 ('Market Street', 20),
 ('Park Avenue', 20),
 ('Washington Street', 19),
 ('Jefferson Street', 19),
 ('Jackson Street', 19),
 ('Oak Street', 19),
 ('Walnut Street', 18),
 ('Elm Street', 18)]
